In [1]:
p = {}
with open("data/displayTrain.txt", "r") as f:
    line = f.readline()
      
    i = 0
    
    while line:
        
        vals = map(int, line[1:-2].split(','))
        for i in range(1,len(vals)):
            if (vals[0], vals[i]) in p:
                p[(vals[0], vals[i])] += 1
            else:
                p[(vals[0], vals[i])] = 1

        line = f.readline();
        i += 1
               
f.close()
        

In [33]:
import pandas as pd
train = pd.read_csv("data/clicks_train.csv", usecols=['ad_id','clicked'])

ad_likelihood = train.groupby('ad_id').clicked.agg(['mean'])

In [50]:
bp = ad_likelihood['mean'].to_dict()

In [66]:
meanAds = ad_likelihood.reset_index()['ad_id'].tolist()

In [71]:
adId1 = pd.read_csv("data/clicks_test.csv", usecols=['ad_id'])
allTestAds = adId1['ad_id'].unique()

del adId1

In [72]:
sAll = set(allTestAds)
sMean = set(meanAds)

sMissing = sAll.difference(sMean)

In [75]:
for i in sMissing:
    bp[i] = 0.0

In [82]:
ads = []
v10 = 0
v01 = 0

j = 0

with open('data/displayTest.txt', 'r') as f:
           
    line = f.readline()
    
    while line:
        
#         j += 1
#         if ((j % 1000) == 0):
#             print("{}".format(j))
     
    
        vals = map(int, line[1:-2].split(','))

        swap = True

       
        while swap:  # Bubble sort

            swap = False
            for i in range(len(vals) - 1):
                
                bp1 = bp[vals[i+1]]
                bp0 = bp[vals[i]]
                
                if (vals[i+1], vals[i]) in p:
                    v10 = p[(vals[i+1], vals[i])] + bp1
                else: v10 = bp1

                if (vals[i], vals[i+1]) in p:
                    v01 = p[(vals[i], vals[i+1])] + bp0
                else: v01 = bp0

                if (v10 > v01):
                    temp = vals[i]
                    vals[i] = vals[i+1]
                    vals[i+1] = temp
                    swap = True
                    
        ads.append(vals)
        
        line = f.readline()
        

In [83]:
import pandas as pd

displayId = pd.read_csv("data/clicks_test.csv", usecols=['display_id'])
displayId = displayId['display_id'].unique()

In [84]:
combined = zip(displayId, ads)

In [85]:
with open('data/advad.csv', 'w') as f:
    
    f.write('display_id,ad_id\n')
    
    for line in combined:
        
        f.write('{},{}\n'.format(line[0], ' '.join(map(str,line[1]))))
    
f.close()